In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random as rn
from matplotlib.patches import ConnectionPatch
import plotly as py

from scipy import stats
import regex as re


seed = 42

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

In [32]:
email=pd.read_csv("spam.csv")
email.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [33]:
email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [34]:
#pd.set_option("display.max_rows",5572)
email["Message"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [35]:
email["spam"]=email["Category"].apply(lambda x: 1 if x=="spam" else 0)
email.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [36]:
email2=email.copy()

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import*
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [38]:
logistic=LogisticRegression()
tree=DecisionTreeClassifier()
forest=RandomForestClassifier()
ada=AdaBoostClassifier()
xgb=XGBClassifier()
#grid_search=GridSearchCV()
clf=MultinomialNB()
svm=SVC()

In [39]:
x1=email2["Message"]
y1=email2["spam"]
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,random_state=50,test_size=0.8)

In [40]:

vectorizer = CountVectorizer()
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])
clf.fit(x_train1,y_train1)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [41]:
encoder=LabelEncoder()
num_col=["Category","Message"]
pd.get_dummies(email,num_col,drop_first=True)
for col in num_col:
    email[col]=encoder.fit_transform(email[col])
email.head()

,Category,Message,spam
0,0,1080,0
1,0,3126,0
2,1,999,1
3,0,4121,0
4,0,2781,0


In [42]:
x=email["Message"]
y=email["spam"]
print(x.shape)
print(y.shape)

(5572,)
(5572,)


In [43]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=50,test_size=0.8)

In [44]:
print(f'X_train size= {x_train.shape}')
print(f'y_train size= {y_train.shape}')

x_train = x_train.values.reshape(-1, 1)
x_test = x_test.values.reshape(-1, 1)
print(f'X_train size= {x_train.shape}')
print(f'x_test size= {x_test.shape}')

print(x.shape)
print(y.shape)

X_train size= (1114,)
y_train size= (1114,)
X_train size= (1114, 1)
x_test size= (4458, 1)
(5572,)
(5572,)


In [45]:
def get_score(model):
    model.fit(x_train,y_train)
    pred=model.predict(x_test)
    accuracy=model.score(x_test,y_test)
    return accuracy


print(f' For Logistic Regression Algorithm, we had an accuracy of {get_score(logistic)}')
print(f' For  Decision Tree Algorithm, we had an accuracy of {get_score(tree)}')
print(f' For Random Forest Algorithm, we had an accuracy of {get_score(forest)}')
print(f' For xgboost Algorithm, we had an accuracy of {get_score(xgb)}')
print(f' For AdaBoostClassifier Algorithm, we had an accuracy of {get_score(ada)}')
print(f' For SVM Algorithm, we had an accuracy of {get_score(svm)}')
print(f' For Naive Bayes Algorithm, we had an accuracy of {clf.score(x_test1,y_test1)}')

 For Logistic Regression Algorithm, we had an accuracy of 0.8694481830417228
 For  Decision Tree Algorithm, we had an accuracy of 0.8660834454912517
 For Random Forest Algorithm, we had an accuracy of 0.8660834454912517
 For xgboost Algorithm, we had an accuracy of 0.8864961866307761
 For AdaBoostClassifier Algorithm, we had an accuracy of 0.8689995513683266
 For SVM Algorithm, we had an accuracy of 0.8694481830417228
 For Naive Bayes Algorithm, we had an accuracy of 0.9800358905338717


In [46]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf,x1,y1)


array([0.98295964, 0.98565022, 0.98294434, 0.98294434, 0.98653501])

In [47]:
x = x.values.reshape(-1, 1)

In [48]:
algorithms = [logistic, tree, forest, ada, xgb, svm]
algorithm_names = ["Logistic Regression", "Decision Tree",
                   "Random Forest", "AdaBoostClassifier",
                   "Xgboost", "SMV"]

# Loop through the algorithms and calculate and print accuracies
for col, col_name in zip(algorithms, algorithm_names):
    accuracies = cross_val_score(col, x, y)
    print(f'The accuracies for {col_name} are: {accuracies}')

The accuracies for Logistic Regression are: [0.86547085 0.86547085 0.86624776 0.86624776 0.86624776]
The accuracies for Decision Tree are: [0.8941704  0.91390135 0.91113106 0.91472172 0.9048474 ]
The accuracies for Random Forest are: [0.8941704  0.91390135 0.91113106 0.91472172 0.9048474 ]
The accuracies for AdaBoostClassifier are: [0.86457399 0.86547085 0.86445242 0.86624776 0.86624776]
The accuracies for Xgboost are: [0.91210762 0.91928251 0.92010772 0.90843806 0.91741472]
The accuracies for SMV are: [0.86547085 0.86547085 0.86624776 0.86624776 0.86624776]


In [49]:
cross_val_score(clf,x_test1,y_test1,cv=100).mean()

0.9872272727272728

Naive Bayes did a great job on this classification problem

# NEURAL NET AND LSTM

In [50]:
df=email2.copy()
df.drop("Category",axis=1,inplace=True)

In [51]:
df.head()

,Message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [52]:

#re
#stem
#stopwords
#ohe
#padseq
#embed and LSTM
#NEUral net

In [53]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import nltk
from nltk.stem import WordNetLemmatizer


In [54]:
ps=PorterStemmer()
pss=WordNetLemmatizer()
corpus=[]
for i in range(0,len(df)):
  review=re.sub("[^a-zA-Z]"," ",df["Message"][i])
  review=nltk.word_tokenize(review)
  review=[pss.lemmatize(word) for word in review if not word in stopwords.words("english")]
  corpus.append(review)


In [55]:
corpus

[['Go',
  'jurong',
  'point',
  'crazy',
  'Available',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'Cine',
  'got',
  'amore',
  'wat'],
 ['Ok', 'lar', 'Joking', 'wif', 'u', 'oni'],
 ['Free',
  'entry',
  'wkly',
  'comp',
  'win',
  'FA',
  'Cup',
  'final',
  'tkts',
  'st',
  'May',
  'Text',
  'FA',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'T',
  'C',
  'apply'],
 ['U', 'dun', 'say', 'early', 'hor', 'U', 'c', 'already', 'say'],
 ['Nah', 'I', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['FreeMsg',
  'Hey',
  'darling',
  'week',
  'word',
  'back',
  'I',
  'like',
  'fun',
  'still',
  'Tb',
  'ok',
  'XxX',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['Even',
  'brother',
  'like',
  'speak',
  'They',
  'treat',
  'like',
  'aid',
  'patent'],
 ['As',
  'per',
  'request',
  'Melle',
  'Melle',
  'Oru',
  'Minnaminunginte',
  'Nurungu',
  'Vettam',
  'set',
  'callertune',
  'Callers',
  'Press',
  'copy',
  'friend',
  'C

In [56]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=corpus, vector_size=500, window=5, min_count=5, workers=4)


In [66]:
model.wv.vocab()

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [58]:
ss6

NameError: name 'ss6' is not defined

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences=pad_sequences(,maxlen=50,padding="post")


In [ ]:
x_train_new=np.array(sequences)
y_train_new=np.array(y)
x_val=x_train_new[5000:]
y_val=y[5000:]

In [ ]:
from tensorflow.keras.layers import*
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
model=Sequential()
#EarlyStopping()
model.add(Embedding(5000,50,input_length=50))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
x_train,x_test,y_train,y_test=train_test_split(x_train_new,y_train_new,random_state=50,test_size=0.5)

In [ ]:
x_train_new.shape,y_train_new.shape

In [ ]:
hist=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=500
         ,batch_size=64)

In [ ]:
y_pred=model.predict(x_test)
y_pred=np.where(y_pred>0.5,1,0)

In [ ]:
from sklearn.metrics import*
conf=confusion_matrix(y_test,y_pred)
conf

In [ ]:
scores=accuracy_score(y_test,y_pred)
scores

In [ ]:
clf=classification_report(y_test,y_pred)
clf

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("Model's Training & Validation loss across epochs")
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()